In [22]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, TimeDistributed, Conv2D, MaxPooling1D, Input, concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import Adam, Nadam
from keras import backend as K

%matplotlib

Using matplotlib backend: Qt5Agg


## Import Tick Data and Create 5min RTH Bars

In [2]:
tick_data = pd.read_feather('../data/processed/ES_tick.feather')
tick_data = tick_data[tick_data['date'] > '2017-07-29']
#Create Index from date column
tick_data.index = tick_data['date']
tick_data.drop(labels=['date'],axis=1,inplace=True)
tick_data.tail()

,last,bid,ask,volume
date,,,,
2018-01-25 19:54:52.612000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:52.615000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:55.332000-05:00,2844.0,2843.75,2844.0,2


In [3]:
#Resample to get 5min bars
five_min_data = pd.DataFrame(
    tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())

import pandas_market_calendars as mcal
#We hack the NYSE Calendar extending the close until 4:15
class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
    @property
    def close_time(self):
        return datetime.time(16, 15)
    
#Create RTH Calendar
nyse = CMERTHCalendar()
schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                         end_date=five_min_data.index.max())

#Filter out those bars that occur during RTH
five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
five_min_data['valid_time'] = False
during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
five_min_data.loc[during_rth, 'valid_time'] = True
five_min_data = five_min_data[five_min_data['valid_time'] == True]
five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)

#Add ema
five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

#Reset index
five_min_data.reset_index(inplace=True)

five_min_data[81:].head()

,date,open,high,low,close,ema
81,2017-08-01 09:35:00-04:00,2475.50,2476.00,2472.50,2473.50,2470.908594
82,2017-08-01 09:40:00-04:00,2473.50,2474.00,2471.50,2472.50,2471.060194
83,2017-08-01 09:45:00-04:00,2472.25,2473.25,2471.75,2473.00,2471.244978
84,2017-08-01 09:50:00-04:00,2473.00,2473.25,2472.00,2472.75,2471.388343
85,2017-08-01 09:55:00-04:00,2472.75,2473.00,2471.25,2471.25,2471.375165


In [83]:
#Add column for number of seconds elapsed in trading day
five_min_data['sec'] = (five_min_data['date'].values 
                        - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

#Calculate sin & cos time
#24hr time is a cyclical continuous feature
seconds_in_day = 24*60*60
five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

five_min_data.drop('sec', axis=1, inplace=True)
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727


## Create Test / Train Datasets

In [107]:
five_min_data = pd.read_feather('../data/processed/ES_TFCnn.feather')
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311,-104,96
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308,-104,96
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873,-104,96
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011,-104,96
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727,-104,96


In [2]:
five_min_data = pd.read_hdf('../data/processed/store.h5', key='cnn_data')
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104


In [3]:
fd = five_min_data
fd['change'] = fd['close'] - fd['close'].shift(1)
fd['cdl_sign'] = np.sign(fd['close'] - fd['open'])
fd['cdl_body'] = np.absolute(fd['close'] - fd['open'])
#fd['cdl_ut'] = fd['high'] - fd['close'] if fd['cdl_body'] > 0 else fd['high'] - fd['open']
fd['cdl_ut'] = np.where(fd['cdl_sign'] > 0, fd['high'] - fd['close'], fd['high'] - fd['open'])
fd['cdl_lt'] = np.where(fd['cdl_sign'] > 0, fd['open'] - fd['low'], fd['close'] - fd['low'])
fd['cdl_rng'] = fd['high'] - fd['low']
fd['cdl_hl'] = np.where(fd['low'] >= fd['low'].shift(), 1, 0) #higher low
fd['cdl_lh'] = np.where(fd['high'] <= fd['high'].shift(), 1, 0) #lower high
fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc,change,cdl_sign,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_hl,cdl_lh
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96,NaN,1.0,0.75,1.00,0.00,1.75,0,0
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96,-0.25,0.0,0.00,0.25,1.00,1.25,0,1
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96,0.50,1.0,0.75,0.25,0.00,1.00,1,0
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96,-1.25,-1.0,1.00,0.25,0.25,1.50,0,1
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104,-1.00,-1.0,1.25,0.75,0.25,2.25,0,1


In [4]:
higher_than_next_bar = fd['high'] > fd.shift(-1)['high']
higher_than_prev_bar = fd['high'] > fd.shift(1)['high']
lower_than_next_bar = fd['low'] < fd.shift(-1)['low']
lower_than_prev_bar = fd['low'] < fd.shift(1)['low']
fd['pivot_high'] = higher_than_next_bar & higher_than_prev_bar
fd['pivot_low'] = lower_than_next_bar & lower_than_prev_bar
fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc,change,cdl_sign,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_hl,cdl_lh,pivot_high,pivot_low
0,2006-01-03 09:35:00-05:00,1259.00,1260.75,1259.00,1259.75,NaN,-0.625923,-0.779884,-104,96,NaN,1.0,0.75,1.00,0.00,1.75,0,0,False,False
1,2006-01-03 09:40:00-05:00,1259.50,1259.75,1258.50,1259.50,NaN,-0.642788,-0.766044,-104,96,-0.25,0.0,0.00,0.25,1.00,1.25,0,1,False,True
2,2006-01-03 09:45:00-05:00,1259.25,1260.25,1259.25,1260.00,NaN,-0.659346,-0.751840,-104,96,0.50,1.0,0.75,0.25,0.00,1.00,1,0,True,False
3,2006-01-03 09:50:00-05:00,1259.75,1260.00,1258.50,1258.75,NaN,-0.675590,-0.737277,-104,96,-1.25,-1.0,1.00,0.25,0.25,1.50,0,1,False,False
4,2006-01-03 09:55:00-05:00,1259.00,1259.75,1257.50,1257.75,NaN,-0.691513,-0.722364,96,-104,-1.00,-1.0,1.25,0.75,0.25,2.25,0,1,False,False


In [5]:
data = fd[81:]

openp = data['open'].tolist()
highp = data['high'].tolist()
lowp = data['low'].tolist()
closep = data['close'].tolist()
emap = data['ema'].tolist()
sin_time = data['sin_time'].tolist()
cos_time = data['cos_time'].tolist()
btc = data['btc'].tolist()
stc = data['stc'].tolist()

change = data['change'].tolist()

cdl_sign = data['cdl_sign'].tolist()
cdl_body = data['cdl_body'].tolist()
cdl_ut = data['cdl_ut'].tolist()
cdl_lt = data['cdl_lt'].tolist()
cdl_rng = data['cdl_rng'].tolist()
cdl_hl = data['cdl_hl'].tolist()
cdl_lh = data['cdl_lh'].tolist()

pivot_high = data['pivot_high'].astype('int').tolist()
pivot_low = data['pivot_low'].astype('int').tolist()

In [58]:
closep = (data['close'].shift(-1) - data['close']).tolist()[:-1]
btc = data['btc'].shift(-1).tolist()[:-1]
sin_time = data['sin_time'].shift(-1).tolist()[:-1]
cos_time = data['cos_time'].shift(-1).tolist()[:-1]

In [173]:
p = int(data.shape[0] * 0.9)
p = 10000
mean = data.mean(axis=0)
std = data.std(axis=0)
mean_c = np.mean(closep[0:p])
std_c = np.std(closep[0:p])
mean_c, std_c


(1288.2096, 21.484784856730588)

In [6]:
WINDOW = 36 #Number of bars in a trading day
EMB_SIZE = 9
STEP = 1
FORECAST = 1

X, Y = [], []
for i in range(0, len(data)-WINDOW+1, STEP):
    try:
        o = openp[i:i+WINDOW]
        h = highp[i:i+WINDOW]
        l = lowp[i:i+WINDOW]
        c = closep[i:i+WINDOW]
        e = emap[i:i+WINDOW]
        ct = cos_time[i:i+WINDOW]
        st = sin_time[i:i+WINDOW]
        
        cng = change[i:i+WINDOW]
        
        _cdl_sign = cdl_sign[i:i+WINDOW]
        _cdl_body = cdl_body[i:i+WINDOW]
        _cdl_ut = cdl_ut[i:i+WINDOW]
        _cdl_lt = cdl_lt[i:i+WINDOW]
        _cdl_rng = cdl_rng[i:i+WINDOW]
        _cdl_hl = cdl_hl[i:i+WINDOW]
        _cdl_lh = cdl_lh[i:i+WINDOW]
        
        _pivot_high = np.array(pivot_high[i:i+WINDOW]) * np.array(h)
        _pivot_low = np.array(pivot_low[i:i+WINDOW]) * np.array(l)
        
        o = (np.array(o) - np.mean(o)) / np.std(o)
        h = (np.array(h) - np.mean(h)) / np.std(h)
        l = (np.array(l) - np.mean(l)) / np.std(l)
        c = (np.array(c) - np.mean(c)) / np.std(c)
        e = (np.array(e) - np.mean(e)) / np.std(e)
        
        ph = (np.array(_pivot_high) - np.mean(h)) / np.std(h)
        pl = (np.array(_pivot_low) - np.mean(l)) / np.std(l)
        
        _cng = (np.array(cng) - np.mean(cng)) / np.std(cng)
        
        
        #c = (np.array(c) - mean_c) / std_c
        
        #o = np.divide(o, c[-1]) 
        #h = np.divide(h, c[-1])
        #l = np.divide(l, c[-1])
        #e = np.divide(e, c[-1])
        #c = np.divide(c, c[-1])

        x_i = closep[i:i+WINDOW]
        y_i = closep[(i+WINDOW-1)+FORECAST]  

        last_close = x_i[-1]
        next_close = y_i

        if btc[i+WINDOW-1] > 0:
            y_i = [1, 0]
        else:
            y_i = [0, 1]
        
        x_i = np.column_stack((o, h, l, c, e, ct, st, cng, _cdl_hl))
        #x_i = np.column_stack((cng))
        #x_i = np.column_stack((c))
        
    except Exception as e:
        #e.throw()
        break

    #only add if 1pt body and close on high
    if (closep[i+WINDOW-1] == highp[i+WINDOW-1]) and (closep[i+WINDOW-1]-openp[i+WINDOW-1]>=1):
        X.append(x_i)
        Y.append(y_i)

In [35]:
X[-1, [1,3]]

array([[ 0.99765046,  0.9986445 ,  0.99746973,  0.99855413,  0.99779185,
        -0.70710678, -0.70710678,  2.5       ,  1.        ],
       [ 0.99855413,  0.99900596,  0.99828303,  0.99882523,  0.99789027,
        -0.69151306, -0.72236396,  0.75      ,  1.        ],
       [ 0.99882523,  0.99909633,  0.9983734 ,  0.99882523,  0.99797931,
        -0.67559021, -0.73727734,  0.        ,  1.        ],
       [ 0.9989156 ,  0.99900596,  0.9983734 ,  0.99855413,  0.99803405,
        -0.65934582, -0.75183981, -0.75      ,  1.        ],
       [ 0.99855413,  0.99873486,  0.9983734 ,  0.9986445 ,  0.99809219,
        -0.64278761, -0.76604444,  0.25      ,  1.        ],
       [ 0.9986445 ,  0.99882523,  0.99746973,  0.99746973,  0.99803291,
        -0.62592347, -0.77988448, -3.25      ,  0.        ],
       [ 0.99746973,  0.99882523,  0.99746973,  0.99882523,  0.99810837,
        -0.60876143, -0.79335334,  3.75      ,  1.        ],
       [ 0.99882523,  0.99900596,  0.99846376,  0.99882523,  0

In [7]:
# Let's split into train and test sets
# Train Set will be from 8/1/17 through 12/31/17, Test Set 1/1/17 - 1/25/17
p = 8547 #Manual split for now
p = int(len(X) * 0.9)
X, Y = np.array(X), np.array(Y)
X_train = X[0:p]
Y_train = Y[0:p]
X_test = X[p:]
Y_test = Y[p:]

#We may want to shuffle the training data -- will look into this later
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

X_train, Y_train = shuffle_in_unison(X_train, Y_train)

# Not sure why this is needed, but we apply it anyway
#X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1], EMB_SIZE))
#X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1], EMB_SIZE))
X_train[-1]

array([[-2.80902677e-01, -5.47949030e-01, -4.35377418e-01,
        -6.73983911e-01, -3.13624790e-01, -1.30526192e-01,
        -9.91444861e-01, -2.50000000e-01,  0.00000000e+00],
       [-9.33321799e-01, -5.47949030e-01, -4.35377418e-01,
        -3.66852002e-01, -3.03958998e-01, -1.08866875e-01,
        -9.94056338e-01,  2.50000000e-01,  1.00000000e+00],
       [-2.80902677e-01, -5.47949030e-01, -8.70754836e-02,
        -5.97200934e-02, -2.25056999e-01, -8.71557427e-02,
        -9.96194698e-01,  2.50000000e-01,  1.00000000e+00],
       [ 4.53068834e-02, -5.47949030e-01, -4.35377418e-01,
        -9.81115820e-01, -3.64139751e-01, -6.54031292e-02,
        -9.97858923e-01, -7.50000000e-01,  0.00000000e+00],
       [-9.33321799e-01, -8.46830318e-01, -4.35377418e-01,
        -6.73983911e-01, -4.19819769e-01, -4.36193874e-02,
        -9.99048222e-01,  2.50000000e-01,  1.00000000e+00],
       [-6.07112238e-01, -5.47949030e-01, -8.70754836e-02,
        -3.66852002e-01, -4.00040169e-01, -2.181488

## Train CNN Model

In [52]:
model = Sequential()

model.add(
    TimeDistributed(
        Conv2D(32, (7, 7), padding='same', strides=2),
        input_shape=(None, 540, 960, 2)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_53 (TimeDis (None, None, 270, 480, 32 4736      
Total params: 4,736
Trainable params: 4,736
Non-trainable params: 0
_________________________________________________________________


In [34]:
model = Sequential()
model.add(TimeDistributed(Convolution1D(filters=32,
                                        kernel_size=2,
                                        padding='same'),
                          input_shape = (None, WINDOW, EMB_SIZE)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.75)))

model.add(TimeDistributed(Convolution1D(filters=64,
                        kernel_size=4,
                        padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.75)))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed((Dense(32))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))

model.add(LSTM(64, recurrent_dropout=0.75))
#model.add(Dropout(0))

model.add(Dense(2))
model.add(Activation('softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_61 (TimeDis (None, None, 36, 32)      160       
_________________________________________________________________
time_distributed_62 (TimeDis (None, None, 36, 32)      128       
_________________________________________________________________
time_distributed_63 (TimeDis (None, None, 36, 32)      0         
_________________________________________________________________
time_distributed_64 (TimeDis (None, None, 36, 32)      0         
_________________________________________________________________
time_distributed_65 (TimeDis (None, None, 36, 64)      8256      
_________________________________________________________________
time_distributed_66 (TimeDis (None, None, 36, 64)      256       
_________________________________________________________________
time_distributed_67 (TimeDis (None, None, 36, 64)      0         
__________

In [14]:
model = Sequential()
model.add(Convolution1D(input_shape = (WINDOW, EMB_SIZE),
                        filters=32,
                        kernel_size=2,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
#model.add(Dropout(0.75))

model.add(Convolution1D(filters=64,
                        kernel_size=4,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
#model.add(Dropout(0.75))

model.add(Flatten())

model.add(Dense(32))
#model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.75))

model.add(Dense(2))
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 36, 32)            608       
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 32)            128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 36, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 36, 64)            8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 36, 64)            256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 36, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
__________

In [90]:
model = Sequential()
model.add(LSTM(64, return_sequences=False,
               input_shape=(WINDOW, EMB_SIZE),
              dropout=0,
              recurrent_dropout=0.75))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, recurrent_dropout=0.75))  # return a single vector of dimension 32
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_30 (LSTM)               (None, 64)                18944     
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 130       
Total params: 19,074
Trainable params: 19,074
Non-trainable params: 0
_________________________________________________________________


In [33]:
cnn_input = Input(shape=(WINDOW, EMB_SIZE), dtype='float32', name='cnn_input')
x = Convolution1D(filters=32,
                  kernel_size=2,
                  padding='same',
                  activation='relu')(cnn_input)
x = Dropout(0.75)(x)
x = Convolution1D(filters=64,
                  kernel_size=4,
                  padding='same',
                  activation='relu')(x)
x = Dropout(0.75)(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)

x1 = LSTM(32, recurrent_dropout=0.75)(cnn_input)

merged = concatenate([x, x1], axis=-1)
x2 = Dense(32, activation='relu')(merged)

output = Dense(2, activation='softmax', name='output')(x2)
model = Model(inputs=[cnn_input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          (None, 36, 9)        0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 36, 32)       608         cnn_input[0][0]                  
__________________________________________________________________________________________________
dropout_14 (Dropout)            (None, 36, 32)       0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 36, 64)       8256        dropout_14[0][0]                 
__________________________________________________________________________________________________
dropout_15

In [34]:
def precision_threshold(threshold=0.5):
    def precision(y_true, y_pred):
        """Precision metric.
        Computes the precision over the whole batch using threshold_value.
        """
        
        y_true = y_true[:, 0]
        y_pred = y_pred[:, 0]
        
        threshold_value = threshold
        # Adaptation of the "round()" used before to get the predictions. Clipping to make sure that the predicted raw values are between 0 and 1.
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        # Compute the number of true positives. Rounding in prevention to make sure we have an integer.
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        # count the predicted positives
        predicted_positives = K.sum(y_pred)
        # Get the precision ratio
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

opt = Nadam(lr=0.0003)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="model.hdf5", verbose=1, save_best_only=True)


model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy', precision_threshold(0.55), precision_threshold(0.6)])

history = model.fit(X_train, Y_train, 
          epochs = 200, 
          batch_size = 64, 
          verbose=1, 
          validation_data=(X_test, Y_test),
          callbacks=[reduce_lr, checkpointer],
          shuffle=True)

Train on 9571 samples, validate on 1064 samples
Epoch 1/200
9571/9571 [==============================] - 6s 583us/step - loss: 0.7244 - acc: 0.5266 - precision: 0.4488 - precision_1: 0.4112 - val_loss: 0.6899 - val_acc: 0.5498 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.68992, saving model to model.hdf5
Epoch 2/200
9571/9571 [==============================] - 4s 425us/step - loss: 0.6964 - acc: 0.5331 - precision: 0.4517 - precision_1: 0.3280 - val_loss: 0.6907 - val_acc: 0.5498 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.68992
Epoch 3/200
9571/9571 [==============================] - 4s 433us/step - loss: 0.6936 - acc: 0.5375 - precision: 0.3605 - precision_1: 0.1337 - val_loss: 0.6900 - val_acc: 0.5498 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.68992
Epoch 4/200
9571/9571 [==============================] - 

9571/9571 [==============================] - 4s 414us/step - loss: 0.6865 - acc: 0.5543 - precision: 0.0201 - precision_1: 0.0134 - val_loss: 0.6893 - val_acc: 0.5498 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00028: val_loss did not improve from 0.68870
Epoch 29/200
9571/9571 [==============================] - 4s 408us/step - loss: 0.6872 - acc: 0.5547 - precision: 0.0301 - precision_1: 0.0000e+00 - val_loss: 0.6890 - val_acc: 0.5498 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00029: val_loss did not improve from 0.68870
Epoch 30/200
9571/9571 [==============================] - 4s 409us/step - loss: 0.6865 - acc: 0.5548 - precision: 0.0134 - precision_1: 0.0067 - val_loss: 0.6892 - val_acc: 0.5470 - val_precision: 0.0000e+00 - val_precision_1: 0.0000e+00

Epoch 00030: val_loss did not improve from 0.68870
Epoch 31/200
9571/9571 [==============================] - 4s 414us/step - loss: 0.6857 - acc: 0.5549 - precision: 0.0067 - precision_1: 0.0

9571/9571 [==============================] - 4s 415us/step - loss: 0.6849 - acc: 0.5564 - precision: 0.2374 - precision_1: 0.0468 - val_loss: 0.6871 - val_acc: 0.5583 - val_precision: 0.3158 - val_precision_1: 0.0451

Epoch 00056: val_loss improved from 0.68728 to 0.68715, saving model to model.hdf5
Epoch 57/200
9571/9571 [==============================] - 4s 430us/step - loss: 0.6846 - acc: 0.5533 - precision: 0.2233 - precision_1: 0.0802 - val_loss: 0.6877 - val_acc: 0.5470 - val_precision: 0.3158 - val_precision_1: 0.0451

Epoch 00057: val_loss did not improve from 0.68715
Epoch 58/200
9571/9571 [==============================] - 4s 427us/step - loss: 0.6833 - acc: 0.5591 - precision: 0.2374 - precision_1: 0.0602 - val_loss: 0.6871 - val_acc: 0.5526 - val_precision: 0.2256 - val_precision_1: 0.0301

Epoch 00058: val_loss improved from 0.68715 to 0.68707, saving model to model.hdf5
Epoch 59/200
9571/9571 [==============================] - 4s 425us/step - loss: 0.6840 - acc: 0.5567 - 

9571/9571 [==============================] - 4s 385us/step - loss: 0.6816 - acc: 0.5593 - precision: 0.3208 - precision_1: 0.1407 - val_loss: 0.6873 - val_acc: 0.5470 - val_precision: 0.3419 - val_precision_1: 0.2256

Epoch 00084: val_loss did not improve from 0.68620
Epoch 85/200
9571/9571 [==============================] - 4s 383us/step - loss: 0.6819 - acc: 0.5589 - precision: 0.3358 - precision_1: 0.1772 - val_loss: 0.6866 - val_acc: 0.5536 - val_precision: 0.4246 - val_precision_1: 0.2767

Epoch 00085: val_loss did not improve from 0.68620
Epoch 86/200
9571/9571 [==============================] - 4s 385us/step - loss: 0.6802 - acc: 0.5575 - precision: 0.4272 - precision_1: 0.2039 - val_loss: 0.6870 - val_acc: 0.5461 - val_precision: 0.4246 - val_precision_1: 0.2767

Epoch 00086: ReduceLROnPlateau reducing learning rate to 0.00024300001678057015.

Epoch 00086: val_loss did not improve from 0.68620
Epoch 87/200
9571/9571 [==============================] - 4s 385us/step - loss: 0.680


Epoch 00112: val_loss did not improve from 0.68620
Epoch 113/200
9571/9571 [==============================] - 4s 387us/step - loss: 0.6769 - acc: 0.5701 - precision: 0.5387 - precision_1: 0.2808 - val_loss: 0.6884 - val_acc: 0.5508 - val_precision: 0.4506 - val_precision_1: 0.3769

Epoch 00113: val_loss did not improve from 0.68620
Epoch 114/200
9571/9571 [==============================] - 4s 422us/step - loss: 0.6783 - acc: 0.5703 - precision: 0.4809 - precision_1: 0.2882 - val_loss: 0.6884 - val_acc: 0.5479 - val_precision: 0.4095 - val_precision_1: 0.3669

Epoch 00114: val_loss did not improve from 0.68620
Epoch 115/200
9571/9571 [==============================] - 4s 466us/step - loss: 0.6785 - acc: 0.5685 - precision: 0.5219 - precision_1: 0.3187 - val_loss: 0.6882 - val_acc: 0.5517 - val_precision: 0.4513 - val_precision_1: 0.4145

Epoch 00115: val_loss did not improve from 0.68620
Epoch 116/200
9571/9571 [==============================] - 4s 432us/step - loss: 0.6782 - acc: 0.56


Epoch 00141: val_loss did not improve from 0.68620
Epoch 142/200
9571/9571 [==============================] - 4s 416us/step - loss: 0.6766 - acc: 0.5693 - precision: 0.5607 - precision_1: 0.4625 - val_loss: 0.6899 - val_acc: 0.5564 - val_precision: 0.5345 - val_precision_1: 0.2506

Epoch 00142: val_loss did not improve from 0.68620
Epoch 143/200
9571/9571 [==============================] - 4s 435us/step - loss: 0.6741 - acc: 0.5715 - precision: 0.5807 - precision_1: 0.4129 - val_loss: 0.6906 - val_acc: 0.5517 - val_precision: 0.5722 - val_precision_1: 0.2807

Epoch 00143: val_loss did not improve from 0.68620
Epoch 144/200
9571/9571 [==============================] - 4s 420us/step - loss: 0.6752 - acc: 0.5754 - precision: 0.5548 - precision_1: 0.4413 - val_loss: 0.6905 - val_acc: 0.5498 - val_precision: 0.5654 - val_precision_1: 0.2907

Epoch 00144: val_loss did not improve from 0.68620
Epoch 145/200
9571/9571 [==============================] - 4s 420us/step - loss: 0.6748 - acc: 0.57

9571/9571 [==============================] - 4s 386us/step - loss: 0.6712 - acc: 0.5817 - precision: 0.6122 - precision_1: 0.5510 - val_loss: 0.6932 - val_acc: 0.5461 - val_precision: 0.5712 - val_precision_1: 0.4326

Epoch 00170: val_loss did not improve from 0.68620
Epoch 171/200
9571/9571 [==============================] - 4s 391us/step - loss: 0.6695 - acc: 0.5813 - precision: 0.5901 - precision_1: 0.5924 - val_loss: 0.6933 - val_acc: 0.5432 - val_precision: 0.5441 - val_precision_1: 0.3366

Epoch 00171: val_loss did not improve from 0.68620
Epoch 172/200
9571/9571 [==============================] - 4s 421us/step - loss: 0.6714 - acc: 0.5796 - precision: 0.5976 - precision_1: 0.5578 - val_loss: 0.6929 - val_acc: 0.5357 - val_precision: 0.5865 - val_precision_1: 0.2764

Epoch 00172: val_loss did not improve from 0.68620
Epoch 173/200
9571/9571 [==============================] - 4s 391us/step - loss: 0.6693 - acc: 0.5830 - precision: 0.6063 - precision_1: 0.5832 - val_loss: 0.6928 - 


Epoch 00198: val_loss did not improve from 0.68620
Epoch 199/200
9571/9571 [==============================] - 4s 408us/step - loss: 0.6657 - acc: 0.5902 - precision: 0.6058 - precision_1: 0.6174 - val_loss: 0.6956 - val_acc: 0.5385 - val_precision: 0.5450 - val_precision_1: 0.4876

Epoch 00199: val_loss did not improve from 0.68620
Epoch 200/200
9571/9571 [==============================] - 4s 406us/step - loss: 0.6639 - acc: 0.5944 - precision: 0.6246 - precision_1: 0.6288 - val_loss: 0.6957 - val_acc: 0.5442 - val_precision: 0.5609 - val_precision_1: 0.4729

Epoch 00200: val_loss did not improve from 0.68620


In [35]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [36]:
from sklearn.metrics import confusion_matrix

#model.load_weights("model.hdf5")
pred = model.predict(np.array(X_test), batch_size=64)

C = confusion_matrix([np.argmax(y) for y in Y_test], [np.argmax(y) for y in pred])

print (C / C.astype(np.float).sum(axis=1)[:, None])

[[0.33264463 0.66735537]
 [0.27931034 0.72068966]]


In [37]:
C

array([[161, 323],
       [162, 418]])

In [39]:
K.clip(pred[0] * Y_test[0], 0, 1)

AttributeError: 'numpy.dtype' object has no attribute 'base_dtype'

In [136]:
sum([np.argmax(y) for y in Y_test])
pd.DataFrame(pred)[0].plot()

In [28]:
data.iloc[43650,:]
pred.shape

(227844, 2)

In [132]:
pred = model.predict(np.array(X_train), batch_size=64)
df = pd.DataFrame(np.concatenate((pred, Y_train), axis=1))
df[(df[0]>.6)&(df[2]==1)].shape
df[(df[0]>.6)].shape

#df.shape
df.sort_values(0,axis=0,ascending=True).head(20)

,0,1,2,3
8,0.094520,0.905480,0.0,1.0
2,0.126728,0.873272,0.0,1.0
3,0.283482,0.716518,0.0,1.0
1,0.372990,0.627011,0.0,1.0
7,0.404254,0.595747,0.0,1.0
5,0.418198,0.581802,1.0,0.0
9,0.773934,0.226066,1.0,0.0
0,0.938506,0.061494,1.0,0.0
4,0.995280,0.004720,1.0,0.0
6,0.997012,0.002988,1.0,0.0


In [133]:
X_train

array([[[-6.85796610e-01, -6.87730965e-01, -6.86905089e-01,
         -6.87830125e-01, -6.88255899e-01, -7.22363962e-01,
         -6.91513056e-01],
        [-6.88337308e-01, -6.89255374e-01, -6.87921389e-01,
         -6.88846406e-01, -6.88309303e-01, -7.07106781e-01,
         -7.07106781e-01],
        [-6.88845448e-01, -6.88239102e-01, -6.87413239e-01,
         -6.87321985e-01, -6.88212421e-01, -6.91513056e-01,
         -7.22363962e-01],
        [-6.86812889e-01, -6.83665874e-01, -6.85380640e-01,
         -6.83765003e-01, -6.87785964e-01, -6.75590208e-01,
         -7.37277337e-01],
        [-6.83764052e-01, -6.85190283e-01, -6.84364341e-01,
         -6.85289424e-01, -6.87545322e-01, -6.59345815e-01,
         -7.51839807e-01],
        [-6.85796610e-01, -6.86206556e-01, -6.84364341e-01,
         -6.85289424e-01, -6.87327599e-01, -6.42787610e-01,
         -7.66044443e-01],
        [-6.85288470e-01, -6.85190283e-01, -6.84364341e-01,
         -6.84781284e-01, -6.87082211e-01, -6.25923472e-01

In [45]:
C / C.astype(np.float).sum(axis=1)[:, None]

array([[0.83673469, 0.15855573, 0.00470958],
       [0.78915663, 0.20481928, 0.0060241 ],
       [0.86335404, 0.13043478, 0.00621118]])

In [69]:
probs = Y_train.sum(axis=0) / Y_train.shape[0]
probs

array([0.55539956, 0.44460044])

In [68]:
pred

array([[0.4436597 , 0.5563404 ],
       [0.5040323 , 0.49596766],
       [0.5359964 , 0.4640036 ],
       ...,
       [0.57311577, 0.4268842 ],
       [0.5362818 , 0.46371824],
       [0.5139088 , 0.48609126]], dtype=float32)

In [120]:
s = np.random.binomial(1, probs[1], pred.shape[0])
s

array([1, 0, 1, ..., 0, 0, 1])

In [121]:
C1 = confusion_matrix([np.argmax(y) for y in Y_test], s)
print (C1 / C1.astype(np.float).sum(axis=1)[:, None])

[[0.58421851 0.41578149]
 [0.59026688 0.40973312]]


In [122]:
([np.argmax(y) for y in Y_test] == s).sum() / pred.shape[0]

0.4984567901234568